In [ ]:
import pandas as pd
import numpy as np
import json
from shapely.geometry import Point, Polygon
from itertools import combinations

# =========================================================
# CONFIGURATION
# =========================================================
FPS = 2
WINDOW_SECONDS = 5
STEP_SECONDS = 2.5

PERSON_CLASSES = ["person"]
MACHINE_CLASSES = ["machinery", "vehicle"]
EPI_POSITIVE = ["hardhat", "safety_vest", "mask"]
EPI_NEGATIVE = ["no-hardhat", "no-safety vest", "no-mask"]

IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080
IMAGE_AREA = IMAGE_WIDTH * IMAGE_HEIGHT


# =========================================================
# UTILS
# =========================================================
def bbox_center(row):
    return np.array([row.bbox_x + row.bbox_w / 2,
                     row.bbox_y + row.bbox_h / 2])


def euclidean(p1, p2):
    return np.linalg.norm(p1 - p2)


def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0]+boxA[2], boxB[0]+boxB[2])
    yB = min(boxA[1]+boxA[3], boxB[1]+boxB[3])
    inter = max(0, xB-xA) * max(0, yB-yA)
    union = boxA[2]*boxA[3] + boxB[2]*boxB[3] - inter
    return inter / union if union > 0 else 0


# =========================================================
# LOAD DATA (detections, zones, plans)
# =========================================================
detections = pd.read_sql("SELECT * FROM detections", con)
zones = pd.read_sql("SELECT * FROM zones WHERE is_active = 1", con)
plans = pd.read_sql("SELECT * FROM plans", con)

detections["timestamp"] = pd.to_datetime(detections["timestamp"])
detections = detections.sort_values("timestamp")


# =========================================================
# PREPARE ZONES
# =========================================================
zones["polygon_shape"] = zones["polygon"].apply(
    lambda p: Polygon(json.loads(p)) if p not in ["{}", None] else None
)


# =========================================================
# WINDOW GENERATION
# =========================================================
windows = []
for cam_id, cam_df in detections.groupby("camera_id"):
    t_min, t_max = cam_df.timestamp.min(), cam_df.timestamp.max()
    t = t_min
    while t + pd.Timedelta(seconds=WINDOW_SECONDS) <= t_max:
        windows.append({
            "camera_id": cam_id,
            "start_time": t,
            "end_time": t + pd.Timedelta(seconds=WINDOW_SECONDS)
        })
        t += pd.Timedelta(seconds=STEP_SECONDS)

windows_df = pd.DataFrame(windows)


# =========================================================
# FEATURE EXTRACTION
# =========================================================
activity_rows = []
risk_rows = []

for _, win in windows_df.iterrows():
    win_df = detections[
        (detections.camera_id == win.camera_id) &
        (detections.timestamp >= win.start_time) &
        (detections.timestamp <= win.end_time)
    ]

    persons = win_df[win_df.object_class == "person"]
    machines = win_df[win_df.object_class.isin(MACHINE_CLASSES)]

    # -------------------------
    # A. Présence humaine
    # -------------------------
    num_persons = persons.groupby("timestamp").size().mean() if not persons.empty else 0
    max_persons = persons.groupby("timestamp").size().max() if not persons.empty else 0
    unique_tracks = persons.track_id.nunique()
    person_density = num_persons / IMAGE_AREA

    # -------------------------
    # B. EPI compliance
    # -------------------------
    helmet_ok = win_df[win_df.object_class == "hardhat"].track_id.nunique()
    helmet_no = win_df[win_df.object_class == "no-hardhat"].track_id.nunique()
    helmet_ratio = helmet_ok / max(helmet_ok + helmet_no, 1)

    vest_ok = win_df[win_df.object_class == "safety_vest"].track_id.nunique()
    vest_no = win_df[win_df.object_class == "no-safety vest"].track_id.nunique()
    vest_ratio = vest_ok / max(vest_ok + vest_no, 1)

    # -------------------------
    # C. Machines
    # -------------------------
    num_machines = machines.track_id.nunique()
    machine_presence_ratio = machines.timestamp.nunique() / max(win_df.timestamp.nunique(), 1)

    # -------------------------
    # D. Distances
    # -------------------------
    min_pm_dist = np.inf
    overlaps_pm = 0

    for _, p in persons.iterrows():
        p_center = bbox_center(p)
        for _, m in machines.iterrows():
            m_center = bbox_center(m)
            min_pm_dist = min(min_pm_dist, euclidean(p_center, m_center))
            if compute_iou(
                (p.bbox_x, p.bbox_y, p.bbox_w, p.bbox_h),
                (m.bbox_x, m.bbox_y, m.bbox_w, m.bbox_h)
            ) > 0.1:
                overlaps_pm += 1

    if min_pm_dist == np.inf:
        min_pm_dist = 0

    # -------------------------
    # E. Zones
    # -------------------------
    persons_in_zone = 0
    time_in_zone = 0
    zone_types = set()
    zone_risk_levels = set()

    for _, z in zones.iterrows():
        if z.polygon_shape is None:
            continue
        for _, p in persons.iterrows():
            pt = Point(bbox_center(p))
            if z.polygon_shape.contains(pt):
                persons_in_zone += 1
                zone_types.add(z.type)
                zone_risk_levels.add(z.risk_level)

    # -------------------------
    # F. Temporal dynamics
    # -------------------------
    speeds = []
    for tid, track in persons.groupby("track_id"):
        track = track.sort_values("timestamp")
        centers = track.apply(bbox_center, axis=1).tolist()
        for c1, c2 in zip(centers[:-1], centers[1:]):
            speeds.append(euclidean(c1, c2))

    avg_speed = np.mean(speeds) if speeds else 0
    std_speed = np.std(speeds) if speeds else 0
    track_lifetime_mean = persons.groupby("track_id").size().mean() if not persons.empty else 0

    # =====================================================
    # DATAFRAMES
    # =====================================================
    activity_rows.append({
        "camera_id": win.camera_id,
        "start_time": win.start_time,
        "end_time": win.end_time,
        "num_persons": num_persons,
        "max_persons": max_persons,
        "unique_tracks": unique_tracks,
        "person_density": person_density,
        "avg_speed_person": avg_speed,
        "std_speed_person": std_speed,
        "num_machines": num_machines,
        "machine_presence_ratio": machine_presence_ratio
    })

    risk_rows.append({
        **activity_rows[-1],
        "helmet_compliance_ratio": helmet_ratio,
        "vest_compliance_ratio": vest_ratio,
        "min_person_machine_distance": min_pm_dist,
        "overlap_person_machine": overlaps_pm,
        "num_persons_in_risk_zone": persons_in_zone,
        "zone_types": list(zone_types),
        "zone_risk_levels": list(zone_risk_levels),
        "track_lifetime_mean": track_lifetime_mean
    })


# =========================================================
# FINAL DATAFRAMES
# =========================================================
df_activity = pd.DataFrame(activity_rows)
df_risk = pd.DataFrame(risk_rows)

print(" Activity features:", df_activity.shape)
print(" Risk features:", df_risk.shape)


In [2]:
!pip install itertools

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools


db/repositories/
├── detection_repo.py
├── plan_repo.py
├── zone_repo.py
├── camera_repo.py
└── alert_repo.py

Pour interférence. 